In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [8]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [11]:
# load the dataset
df = pd.read_csv("/content/drive/MyDrive/Machine_Asignment2_dataset/weather_forecast_data.csv")


In [12]:
# Task 1: Preprocessing
# 1. Check for missing values
def checkMissing(df):
    print("Missing values per column:")
    print(df.isnull().sum())

In [13]:
# 2. Handle missing values by two techniques
def handle_missing(df, strategy):
    if strategy == "drop":
        return df.dropna()
    elif strategy == "replace":
        df_copy = df.copy()
        # Fill numeric columns with mean
        for col in df_copy.select_dtypes(include="number").columns:
            df_copy[col] = df_copy[col].fillna(df_copy[col].mean())
        # Fill categorical columns with mode
        for col in df_copy.select_dtypes(include="object").columns:
            df_copy[col] = df_copy[col].fillna(df_copy[col].mode()[0])
        return df_copy
    else:
        raise ValueError("Invalid missing value strategy")

In [14]:
# 3. Preprocessing
def preprocess(df, scaling):
    X = df.drop(columns=["Rain"])
    y = df["Rain"]

    # Encode target
    y = LabelEncoder().fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale numeric features
    scaler = MinMaxScaler() if scaling == "min-max" else StandardScaler()
    toscale = X_train.select_dtypes(include="number").columns
    X_train[toscale] = scaler.fit_transform(X_train[toscale])
    X_test[toscale] = scaler.transform(X_test[toscale])

    return X_train, X_test, y_train, y_test

In [15]:
# 4. Evaluate model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    return accuracy, precision, recall

In [26]:
#naive bayes sklearn
from sklearn.naive_bayes import GaussianNB

def naive_bayes(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    return evaluate_model(model, X_test, y_test)


In [27]:
def compareStrategies(df):
    strategies = ["drop", "replace"]
    for strategy in strategies:
      print(f"Missing Values Strategy: {strategy}")
      df_handled = handle_missing(df, strategy)

      X_train, X_test, y_train, y_test = preprocess(df_handled, "standard")

      accuracy, precision, recall = naive_bayes(X_train, y_train, X_test, y_test)

      print(f"Accuracy: {accuracy}")
      print(f"Precision: {precision}")
      print(f"Recall: {recall}")
      print()


checkMissing(df)
print()

compareStrategies(df)

Missing values per column:
Temperature    25
Humidity       40
Wind_Speed     32
Cloud_Cover    33
Pressure       27
Rain            0
dtype: int64

Missing Values Strategy: drop
Accuracy: 0.9617021276595744
Precision: 1.0
Recall: 0.7352941176470589

Missing Values Strategy: replace
Accuracy: 0.964
Precision: 1.0
Recall: 0.6785714285714286

